# DeepDive Tutorial <small>Extracting mentions of spouses from the news</small>

In this tutorial, we show an example of a prototypical task that DeepDive is often applied to:
extraction of _structured information_ from _unstructured or 'dark' data_ such as web pages, text documents, images, etc.
While DeepDive can be used as a more general platform for statistical learning and data processing, most of the tooling described herein has been built for this type of use case, based on our experience of successfully applying DeepDive to [a variety of real-world problems of this type](http://deepdive.stanford.edu/showcase/apps).

In this setting, our goal is to take in a set of unstructured (and/or structured) inputs, and populate a relational database table with extracted outputs, along with marginal probabilities for each extraction representing DeepDive's confidence in the extraction.
More formally, we write a DeepDive application to extract mentions of _relations_ and their constituent _entities_ or _attributes_, according to a specified schema; this task is often referred to as **_relation extraction_**.*
Accordingly, we'll walk through an example scenario where we wish to extract mentions of two people being spouses from news articles.

The high-level steps we'll follow are:

1. **Data processing.** First, we'll load the raw corpus, add NLP markups, extract a set of _candidate_ relation mentions, and a sparse _feature_ representation of each.

2. **Distant supervision with data and rules.** Next, we'll use various strategies to provide _supervision_ for our dataset, so that we can use machine learning to learn the weights of a model.

3. **Learning and inference: model specification.** Then, we'll specify the high-level configuration of our _model_.

4. **Error analysis and debugging.** Finally, we'll show how to use DeepDive's labeling, error analysis and debugging tools.

*_Note the distinction between extraction of true, i.e., factual, relations and extraction of mentions of relations.
In this tutorial, we do the latter, however DeepDive supports further downstream methods for tackling the former task in a principled manner._


Whenever something isn't clear, you can always refer to [the complete example code at `examples/spouse/`](https://github.com/HazyResearch/deepdive/tree/master/examples/spouse/) that contains everything shown in this document.

## 0. Preparation

### 0.0. Installing DeepDive and tweaking notebook
First of all, let's make sure DeepDive is installed and can be used from this notebook.
See [DeepDive installation guide](http://deepdive.stanford.edu/installation) for more details.

In [1]:
# PATH needs correct setup to use DeepDive
import os; PWD=os.getcwd(); HOME=os.environ["HOME"]; PATH=os.environ["PATH"]
# home directory installation
%env PATH=$HOME/local/bin:$PATH
# notebook-local installation
%env PATH=$PWD/deepdive/bin:$PATH

!type deepdive
no_deepdive_found = !type deepdive >/dev/null
if no_deepdive_found: # install it next to this notebook
    !bash -c 'PREFIX="$PWD"/deepdive bash <(curl -fsSL git.io/getdeepdive) deepdive_from_release'

env: PATH=/ConfinedWater/local/bin:/ConfinedWater/venv/bin:/opt/homebrew/opt/postgresql/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/opt/X11/bin:/usr/local/MacGPG2/bin:/Library/TeX/texbin
env: PATH=/ConfinedWater/spouse/deepdive/bin:/ConfinedWater/venv/bin:/opt/homebrew/opt/postgresql/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/opt/X11/bin:/usr/local/MacGPG2/bin:/Library/TeX/texbin
deepdive is /ConfinedWater/spouse/deepdive/bin/deepdive


We need to make sure this IPython/Jupyter notebook will work correctly with DeepDive:

In [2]:
# check if notebook kernel was launched in a Unicode locale
import locale; LC_CTYPE = locale.getpreferredencoding()
if LC_CTYPE != "UTF-8":
    raise EnvironmentError("Notebook is running in '%s' encoding not compatible with DeepDive's Unicode output.\n\nPlease restart notebook in a UTF-8 locale with a command like the following:\n\n    LC_ALL=en_US.UTF-8 jupyter notebook" % (LC_CTYPE))

### 0.1. Declaring what to predict

Above all, we shall tell DeepDive what we want to predict as a *random variable* in a language called *DDlog*, stored in a file `app.ddlog`:

In [3]:
%%file app.ddlog
## Random variable to predict #################################################

# This application's goal is to predict whether a given pair of person mention
# are indicating a spouse relationship or not.
has_spouse?(
    p1_id text,
    p2_id text
).

Overwriting app.ddlog


In this notebook, we are going to write our application in this `app.ddlog` one part at a time.
We can check if the code make sense by asking DeepDive to compile it.
DeepDive automatically compiles our application whenever we execute things after making changes, but we can also do this manually by running:

In [4]:
!deepdive compile

2016-07-08 05:46:41.894173 ‘run/LATEST.COMPILE’ -> ‘20160708/054641.839249000’
2016-07-08 05:46:41.894334 ‘run/RUNNING.COMPILE’ -> ‘20160708/054641.839249000’
2016-07-08 05:46:41.894350 Parsing DeepDive application (/ConfinedWater/spouse) to generate:
2016-07-08 05:46:41.894361  run/compiled/deepdive.conf
2016-07-08 05:46:41.894370   from app.ddlog
2016-07-08 05:46:42.253049  run/compiled/deepdive.conf.json
2016-07-08 05:46:42.339479 Performing sanity checks on run/compiled/deepdive.conf.json:
2016-07-08 05:46:42.368615  checking if input_extractors_well_defined
2016-07-08 05:46:42.368663  checking if input_schema_wellformed
2016-07-08 05:46:42.368988 Normalizing and adding built-in processes to the data flow to compile:
2016-07-08 05:46:42.369383  run/compiled/config-0.00-init_objects.json
2016-07-08 05:46:42.382592  run/compiled/config-0.01-parse_calibration.json
2016-07-08 05:46:42.394131  run/compiled/config-0.01-parse_schema.json
2016-07-08 05:46:42.412837  run/compiled/config-0.5

### 0.2. Setting up a database

Next, DeepDive will store all data—input, intermediate, output, etc.—in a relational database.
Currently, Postgres and Greenplum are supported.
For operating at a larger scale, Greenplum is strongly recommended.
To set the location of this database, we need to configure a URL in the [`db.url` file](../examples/spouse/db.url), e.g.:

In [5]:
!echo 'postgresql://localhost/deepdive_spouse_$USER' >db.url

_Note: DeepDive will drop and then create this database if run from scratch—beware of pointing to an existing populated one!_

In [6]:
!deepdive redo init/app

‘process/init/app.done’ -> ‘process/init/app.done~’
‘run/RUNNING’ -> ‘20160708/054643.579621000’
‘run/LATEST’ -> ‘20160708/054643.579621000’
2016-07-08 05:46:43.727087 #!/bin/sh
2016-07-08 05:46:43.727266 # Host: scuba.local
2016-07-08 05:46:43.727290 # App: /ConfinedWater/spouse
2016-07-08 05:46:43.727308 # Targets: init/app
2016-07-08 05:46:43.727328 # Plan: run/20160708/054643.579621000/plan.sh
2016-07-08 05:46:43.727347 export DEEPDIVE_PWD='/ConfinedWater/spouse'
2016-07-08 05:46:43.727365 # execution plan for process/init/app
2016-07-08 05:46:43.727383 
2016-07-08 05:46:43.727400 ## process/init/app ##########################################################
2016-07-08 05:46:43.727416 # Done: 2016-07-08T05:43:15-0700 (3m 28s ago)
2016-07-08 05:46:43.727432 process/init/app/run.sh
2016-07-08 05:46:43.727452 ++ dirname process/init/app/run.sh
2016-07-08 05:46:43.727472 + cd process/init/app
2016-07-08 05:46:43.727490 + export DEEPDIVE_CURRENT_PROCESS_NAME=process/init/app
2016-07-08 

## 1. Data processing

In this section, we'll generate the traditional inputs of a statistical learning-type problem: candidate spouse relations, represented by a set of features, which we will aim to classify as _actual_ relation mentions or not.

We'll do this in four basic steps:

1. Loading raw input data
2. Adding NLP markups
3. Extracting candidate relation mentions
4. Extracting features for each candidate



### 1.1. Loading raw input data
Our first task is to download and load the raw text of [a corpus of news articles provided by Signal Media](http://research.signalmedia.co/newsir16/signal-dataset.html) into an `articles` table in our database.

Keeping the identifier of each article and its content in the table would be good enough.
We can tell DeepDive to do this by declaring the schema of this `articles` table in our `app.ddlog` file; we add the following lines:

In [7]:
%%file -a app.ddlog

## Input Data #################################################################
articles(
    id      text,
    content text
).

Appending to app.ddlog


DeepDive can use a script's output as a data source for loading data into the table if we follow a simple naming convention.
We create a simple shell script at `input/articles.tsj.sh` that outputs the news articles in TSJ format (tab-separated JSONs) from the downloaded corpus.

In [8]:
!mkdir -p input

In [9]:
%%file input/articles.tsj.sh
#!/usr/bin/env bash
set -euo pipefail
cd "$(dirname "$0")"

corpus=signalmedia/signalmedia-1m.jsonl
[[ -e "$corpus" ]] || {
    echo "ERROR: Missing $PWD/$corpus"
    echo "# Please Download it from http://research.signalmedia.co/newsir16/signal-dataset.html"
    echo
    echo "# Alternatively, use our sampled data by running:"
    echo "deepdive load articles input/articles-100.tsv.bz2"
    echo
    echo "# Or, skipping all NLP markup processes by running:"
    echo "deepdive create table sentences"
    echo "deepdive load sentences"
    echo "deepdive mark done sentences"
    false
} >&2

cat "$corpus" |
#grep -E 'wife|husband|married' |
#head -100 |
jq -r '[.id, .content] | map(@json) | join("\t")'

Overwriting input/articles.tsj.sh


We need to mark the script as an executable so DeepDive can actually execute it:

In [10]:
!chmod +x input/articles.tsj.sh

The aforementioned script reads a sample of the corpus (provided as lines of JSON objects), and then using the [jq](https://stedolan.github.io/jq/) language extracts the fields `id` (for article identifier) and `content` from each entry and format those into TSJ.
We can uncomment the `grep` or `head` lines in between and apply some naive filter to subsample articles.

Now, we tell DeepDive to execute the steps to load the `articles` table using the `input/articles.tsj.sh` script.
You must have the [full corpus](http://research.signalmedia.co/newsir16/signal-dataset.html) downloaded at `input/signalmedia/signalmedia-1m.jsonl` for the following to finish correctly.

In [11]:
!deepdive redo articles

app.ddlog: updated since last `deepdive compile`
2016-07-08 05:46:44.787573 ‘run/LATEST.COMPILE’ -> ‘20160708/054644.744063000’
2016-07-08 05:46:44.787780 ‘run/RUNNING.COMPILE’ -> ‘20160708/054644.744063000’
2016-07-08 05:46:44.787801 Parsing DeepDive application (/ConfinedWater/spouse) to generate:
2016-07-08 05:46:44.787813  run/compiled/deepdive.conf
2016-07-08 05:46:44.787823   from app.ddlog
2016-07-08 05:46:45.140000  run/compiled/deepdive.conf.json
2016-07-08 05:46:45.254052 Performing sanity checks on run/compiled/deepdive.conf.json:
2016-07-08 05:46:45.282141  checking if input_extractors_well_defined
2016-07-08 05:46:45.282195  checking if input_schema_wellformed
2016-07-08 05:46:45.282436 Normalizing and adding built-in processes to the data flow to compile:
2016-07-08 05:46:45.282800  run/compiled/config-0.00-init_objects.json
2016-07-08 05:46:45.295065  run/compiled/config-0.01-parse_calibration.json
2016-07-08 05:46:45.307284  run/compiled/config-0.01-parse_schema.json
20

Alternatively, a sample of 100 and 1000 articles can be downloaded from GitHub and loaded into DeepDive with the following command:

In [12]:
NUM_ARTICLES = 100
ARTICLES_FILE = "articles-%d.tsj.bz2" % NUM_ARTICLES

articles_not_done = !deepdive done articles || date
if articles_not_done:
    !cd input && curl -RLO "https://github.com/HazyResearch/deepdive/raw/tab-separated-jsons/examples/spouse/input/"$ARTICLES_FILE
    !deepdive reload articles input/$ARTICLES_FILE

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   184  100   184    0     0    390      0 --:--:-- --:--:-- --:--:--   391
100  135k  100  135k    0     0   123k      0  0:00:01  0:00:01 --:--:--  275k
CREATE TABLE
Loading articles from input/articles-100.tsj.bz2 (tsj format)
COPY 100


After DeepDive finishes creating the table and then fetching and loading the data, we can take a look at the loaded data using the following `deepdive query` command, which enumerates the values for the `id` column of the `articles` table:

In [13]:
!deepdive query '|10 ?- articles(id, _).'

                  id                  
--------------------------------------
 ba44d0cd-bff2-4875-8036-86f37419b5e7
 c5f8a528-cc0f-4f3e-aaef-b9e3b6b00325
 0d07e617-00d4-4866-aee2-0ae197ae366f
 ebcd41ea-e5b4-43a4-9e16-4406d81cfcda
 7516303b-0db5-477d-9e5d-243a73865e39
 f6e047d0-e409-42a6-ab0e-13ab926719a6
 15d53efb-2151-4164-aee0-cae51faedeeb
 fe6e8fcc-1128-4410-923d-f05c42174336
 8b31ede3-0f3b-431a-86a3-342ee18cfd83
 4336860e-fa87-4f54-b3ce-b4afb72c4acd
(10 rows)



### 1.2. Adding NLP markups
Next, we'll use Stanford's [CoreNLP](http://stanfordnlp.github.io/CoreNLP/) natural language processing (NLP) system to add useful markups and structure to our input data.
This step will split up our articles into sentences and their component _tokens_ (roughly, the words).
Additionally, we'll get _lemmas_ (normalized word forms), _part-of-speech (POS) tags_, _named entity recognition (NER) tags_, and a dependency parse of the sentence.

Let's first declare the output schema of this step in `app.ddlog`:

In [14]:
%%file -a app.ddlog

## NLP markup #################################################################
sentences(
    doc_id         text,
    sentence_index int,
    tokens         json,
    lemmas         json,
    pos_tags       json,
    ner_tags       json,
    doc_offsets    json,
    dep_types      json,
    dep_tokens     json
).


Appending to app.ddlog


Next, we declare a DDlog function which takes in the `doc_id` and `content` for an article and returns rows conforming to the sentences schema we just declared, using the **user-defined function (UDF)** in `udf/nlp_markup.sh`.
We specify that this `nlp_markup` function should be run over each row from `articles`, and the output appended to `sentences`:

In [15]:
%%file -a app.ddlog

function nlp_markup over (
        doc_id  text,
        content text
    ) returns rows like sentences
    implementation "udf/nlp_markup.sh" handles tsj lines.

sentences += nlp_markup(doc_id, content) :-
    articles(doc_id, content).


Appending to app.ddlog


This UDF `udf/nlp_markup.sh` is a Bash script which uses [our own wrapper around CoreNLP](https://github.com/HazyResearch/deepdive/tree/deepdive-corenlp/util/nlp).

In [16]:
!mkdir -p udf

In [17]:
%%file udf/nlp_markup.sh
#!/usr/bin/env bash
# Parse documents in tab-separated JSONs input stream with CoreNLP
#
# $ deepdive corenlp install
# $ deepdive corenlp start
# $ deepdive env udf/nlp_markup.sh
# $ deepdive corenlp stop
##
set -euo pipefail
cd "$(dirname "$0")"

# some configuration knobs for CoreNLP
: ${CORENLP_PORT:=$(deepdive corenlp unique-port)}  # a CoreNLP server started ahead of time is shared across parallel UDF processes
# See: http://stanfordnlp.github.io/CoreNLP/annotators.html
: ${CORENLP_ANNOTATORS:="
        tokenize
        ssplit
        pos
        ner
        lemma
        depparse
    "}
export CORENLP_PORT
export CORENLP_ANNOTATORS

# make sure CoreNLP server is available
deepdive corenlp is-running || {
    echo >&2 "PLEASE MAKE SURE YOU HAVE RUN: deepdive corenlp start"
    false
}

# parse input with CoreNLP and output a row for every sentence
deepdive corenlp parse-tsj docid+ content=nlp -- docid nlp |
deepdive corenlp sentences-tsj docid content:nlp \
                            -- docid nlp.{index,tokens.{word,lemma,pos,ner,characterOffsetBegin}} \
                                     nlp.collapsed-dependencies.{dep_type,dep_token}


Overwriting udf/nlp_markup.sh


Again, we mark it as executable for DeepDive to run it:

In [18]:
!chmod +x udf/nlp_markup.sh

Before executing this NLP markup step, we need to launch the CoreNLP server in advance, which may take a while to install and load everything.
Note that the CoreNLP library requires Java 8 to run.

In [19]:
!deepdive corenlp install
!deepdive corenlp start

CoreNLP already installed at /Users/netj/2016/deepdive3/dist/stage/lib/stanford-corenlp/corenlp
CoreNLP server at CORENLP_PORT=10943 already running (PID 38889)
CoreNLP server at CORENLP_PORT=10943 ready.
To stop it after final use, run: deepdive corenlp stop
To watch its log, run: deepdive corenlp watch-log


In [20]:
!deepdive redo sentences

app.ddlog: updated since last `deepdive compile`
2016-07-08 05:46:51.601886 ‘run/LATEST.COMPILE’ -> ‘20160708/054651.549476000’
2016-07-08 05:46:51.602099 ‘run/RUNNING.COMPILE’ -> ‘20160708/054651.549476000’
2016-07-08 05:46:51.602126 Parsing DeepDive application (/ConfinedWater/spouse) to generate:
2016-07-08 05:46:51.602147  run/compiled/deepdive.conf
2016-07-08 05:46:51.602165   from app.ddlog
2016-07-08 05:46:52.075969  run/compiled/deepdive.conf.json
2016-07-08 05:46:52.202008 Performing sanity checks on run/compiled/deepdive.conf.json:
2016-07-08 05:46:52.235002  checking if input_extractors_well_defined
2016-07-08 05:46:52.235091  checking if input_schema_wellformed
2016-07-08 05:46:52.235328 Normalizing and adding built-in processes to the data flow to compile:
2016-07-08 05:46:52.235723  run/compiled/config-0.00-init_objects.json
2016-07-08 05:46:52.248692  run/compiled/config-0.01-parse_calibration.json
2016-07-08 05:46:52.262922  run/compiled/config-0.01-parse_schema.json
20

Now, if we take a look at a sample of the NLP markups, they will have tokens and NER tags that look like the following:

In [21]:
%%bash
deepdive query '
    doc_id, index, tokens, ner_tags | 5
    ?- sentences(doc_id, index, tokens, lemmas, pos_tags, ner_tags, _, _, _).
'

                doc_id                | index |                                                                                                                                                                                                                                tokens                                                                                                                                                                                                                                |                                                                                                                                                               ner_tags                                                                                                                                                                
--------------------------------------+-------+---------------------------------------------------------------------------------------------------------------------------

### 1.3. Extracting candidate relation mentions

#### Mentions of people
Once again we first declare the schema:


In [22]:
%%file -a app.ddlog

## Candidate mapping ##########################################################
person_mention(
    mention_id     text,
    mention_text   text,
    doc_id         text,
    sentence_index int,
    begin_index    int,
    end_index      int
).


Appending to app.ddlog


We will be storing each person as a row referencing a sentence with beginning and ending indexes.
Again, we next declare a function that references a UDF and takes as input the sentence tokens and NER tags:

In [23]:
%%file -a app.ddlog

function map_person_mention over (
        doc_id         text,
        sentence_index int,
        tokens         text[],
        ner_tags       text[]
    ) returns rows like person_mention
    implementation "udf/map_person_mention.py" handles tsj lines.


Appending to app.ddlog


We'll write a simple UDF in Python that will tag spans of contiguous tokens with the NER tag `PERSON` as person mentions (i.e., we'll essentially rely on CoreNLP's NER module).
Note that we've already used a Bash script as a UDF, and indeed any programming language can be used.
(DeepDive will just check the path specified in the top line, e.g., `#!/usr/bin/env python`.)
However, DeepDive provides some convenient utilities for Python UDFs which handle all IO encoding/decoding.
To write our UDF `udf/map_person_mention.py`, we'll start by specifying that our UDF will handle TSV lines (as specified in the DDlog above).
Additionally, we'll specify the exact type schema of both input and output, which DeepDive will check for us:

In [24]:
%%file udf/map_person_mention.py
#!/usr/bin/env python
from deepdive import *

@tsj_extractor
@returns(lambda
        mention_id       = "text",
        mention_text     = "text",
        doc_id           = "text",
        sentence_index   = "int",
        begin_index      = "int",
        end_index        = "int",
    :[])
def extract(
        doc_id         = "text",
        sentence_index = "int",
        tokens         = "text[]",
        ner_tags       = "text[]",
    ):
    """
    Finds phrases that are continuous words tagged with PERSON.
    """
    num_tokens = len(ner_tags)
    # find all first indexes of series of tokens tagged as PERSON
    first_indexes = (i for i in xrange(num_tokens) if ner_tags[i] == "PERSON" and (i == 0 or ner_tags[i-1] != "PERSON"))
    for begin_index in first_indexes:
        # find the end of the PERSON phrase (consecutive tokens tagged as PERSON)
        end_index = begin_index + 1
        while end_index < num_tokens and ner_tags[end_index] == "PERSON":
            end_index += 1
        end_index -= 1
        # generate a mention identifier
        mention_id = "%s_%d_%d_%d" % (doc_id, sentence_index, begin_index, end_index)
        mention_text = " ".join(map(lambda i: tokens[i], xrange(begin_index, end_index + 1)))
        # Output a tuple for each PERSON phrase
        yield [
            mention_id,
            mention_text,
            doc_id,
            sentence_index,
            begin_index,
            end_index,
        ]


Overwriting udf/map_person_mention.py


In [25]:
!chmod +x udf/map_person_mention.py

Above, we write a simple function which extracts and tags all subsequences of tokens having the NER tag "PERSON".
Note that the `extract` function must be a generator (i.e., use a `yield` statement to return output rows).

Finally, we specify that the function will be applied to rows from the `sentences` table and append to the `person_mention` table:

In [26]:
%%file -a app.ddlog

person_mention += map_person_mention(
    doc_id, sentence_index, tokens, ner_tags
) :-
    sentences(doc_id, sentence_index, tokens, _, _, ner_tags, _, _, _).


Appending to app.ddlog


Again, to run, just compile and execute as in previous steps:

In [27]:
!deepdive redo person_mention

app.ddlog: updated since last `deepdive compile`
2016-07-08 05:47:15.232073 ‘run/LATEST.COMPILE’ -> ‘20160708/054715.185040000’
2016-07-08 05:47:15.232309 ‘run/RUNNING.COMPILE’ -> ‘20160708/054715.185040000’
2016-07-08 05:47:15.232326 Parsing DeepDive application (/ConfinedWater/spouse) to generate:
2016-07-08 05:47:15.232338  run/compiled/deepdive.conf
2016-07-08 05:47:15.232348   from app.ddlog
2016-07-08 05:47:15.702262  run/compiled/deepdive.conf.json
2016-07-08 05:47:15.900239 Performing sanity checks on run/compiled/deepdive.conf.json:
2016-07-08 05:47:15.937662  checking if input_extractors_well_defined
2016-07-08 05:47:15.937797  checking if input_schema_wellformed
2016-07-08 05:47:15.938293 Normalizing and adding built-in processes to the data flow to compile:
2016-07-08 05:47:15.938842  run/compiled/config-0.00-init_objects.json
2016-07-08 05:47:15.956245  run/compiled/config-0.01-parse_calibration.json
2016-07-08 05:47:15.972598  run/compiled/config-0.01-parse_schema.json
20

In [28]:
%%bash
deepdive query '
    name, doc, sentence, begin, end | 20
    ?- person_mention(p_id, name, doc, sentence, begin, end).
'

             name             |                 doc                  | sentence | begin | end 
------------------------------+--------------------------------------+----------+-------+-----
 Jackie                       | df13cc43-53fd-4f09-9a7e-d69b12a4adc0 |        1 |     0 |   0
 Joan                         | df13cc43-53fd-4f09-9a7e-d69b12a4adc0 |        2 |     0 |   0
 Jackie Collins               | df13cc43-53fd-4f09-9a7e-d69b12a4adc0 |        2 |     2 |   3
 Graham Norton Show Dame Joan | df13cc43-53fd-4f09-9a7e-d69b12a4adc0 |        2 |    14 |  18
 Jackie                       | df13cc43-53fd-4f09-9a7e-d69b12a4adc0 |       18 |    12 |  12
 Dame Joan                    | df13cc43-53fd-4f09-9a7e-d69b12a4adc0 |       19 |     6 |   7
 Mega Buzz                    | 8b31ede3-0f3b-431a-86a3-342ee18cfd83 |        9 |     6 |   7
 John Dawkins                 | ebcd41ea-e5b4-43a4-9e16-4406d81cfcda |       16 |    24 |  25
 Fatfish                      | ebcd41ea-e5b4-43a4-9e16-44

#### Mentions of spouses (pairs of people)
Next, we'll take all pairs of **non-overlapping person mentions that co-occur in a sentence with less than 5 people total,** and consider these as the set of potential ('candidate') spouse mentions.
We thus filter out sentences with large numbers of people for the purposes of this tutorial; however, these could be included if desired.
Again, to start, we declare the schema for our `spouse_candidate` table—here just the two names, and the two `person_mention` IDs referred to:



In [29]:
%%file -a app.ddlog

spouse_candidate(
    p1_id   text,
    p1_name text,
    p2_id   text,
    p2_name text
).


Appending to app.ddlog


Next, for this operation we don't use any UDF script, instead rely entirely on DDlog operations.
We simply construct a table of person counts, and then do a join with our filtering conditions.
In DDlog this looks like:

In [30]:
%%file -a app.ddlog

num_people(doc_id, sentence_index, COUNT(p)) :-
    person_mention(p, _, doc_id, sentence_index, _, _).

spouse_candidate(p1, p1_name, p2, p2_name) :-
    num_people(same_doc, same_sentence, num_p),
    person_mention(p1, p1_name, same_doc, same_sentence, p1_begin, _),
    person_mention(p2, p2_name, same_doc, same_sentence, p2_begin, _),
    num_p < 5,
    p1 < p2,
    p1_name != p2_name,
    p1_begin != p2_begin.


Appending to app.ddlog


Now, let's tell DeepDive to run what we have so far:

In [31]:
!deepdive redo spouse_candidate

app.ddlog: updated since last `deepdive compile`
2016-07-08 05:47:20.031726 ‘run/LATEST.COMPILE’ -> ‘20160708/054719.981599000’
2016-07-08 05:47:20.031961 ‘run/RUNNING.COMPILE’ -> ‘20160708/054719.981599000’
2016-07-08 05:47:20.031977 Parsing DeepDive application (/ConfinedWater/spouse) to generate:
2016-07-08 05:47:20.031988  run/compiled/deepdive.conf
2016-07-08 05:47:20.031997   from app.ddlog
2016-07-08 05:47:20.551321  run/compiled/deepdive.conf.json
2016-07-08 05:47:20.717768 Performing sanity checks on run/compiled/deepdive.conf.json:
2016-07-08 05:47:20.750829  checking if input_extractors_well_defined
2016-07-08 05:47:20.750882  checking if input_schema_wellformed
2016-07-08 05:47:20.751153 Normalizing and adding built-in processes to the data flow to compile:
2016-07-08 05:47:20.751557  run/compiled/config-0.00-init_objects.json
2016-07-08 05:47:20.767516  run/compiled/config-0.01-parse_calibration.json
2016-07-08 05:47:20.782367  run/compiled/config-0.01-parse_schema.json
20

In [32]:
%%bash
deepdive query '
    name1, name2, doc, sentence | 20
    ?- spouse_candidate(p1, name1, p2, name2),
       person_mention(p1, _, doc, sentence, _, _).
'

            name1             |            name2             |                 doc                  | sentence 
------------------------------+------------------------------+--------------------------------------+----------
 Graham Norton Show Dame Joan | Jackie Collins               | df13cc43-53fd-4f09-9a7e-d69b12a4adc0 |        2
 Joan                         | Jackie Collins               | df13cc43-53fd-4f09-9a7e-d69b12a4adc0 |        2
 Joan                         | Graham Norton Show Dame Joan | df13cc43-53fd-4f09-9a7e-d69b12a4adc0 |        2
 Alex Ellinghausen            | Abbott                       | c5f8a528-cc0f-4f3e-aaef-b9e3b6b00325 |       17
 Lachlan                      | Alicia Metcalfe              | 4336860e-fa87-4f54-b3ce-b4afb72c4acd |        0
 Alicia Metcalfe              | David                        | 4336860e-fa87-4f54-b3ce-b4afb72c4acd |        0
 Lachlan                      | David                        | 4336860e-fa87-4f54-b3ce-b4afb72c4acd |        0

### 1.4. Extracting features for each candidate
Finally, we will extract a set of **features** for each candidate:



In [33]:
%%file -a app.ddlog

## Feature Extraction #########################################################
 
# Feature extraction (using DDLIB via a UDF) at the relation level
spouse_feature(
    p1_id   text,
    p2_id   text,
    feature text
).


Appending to app.ddlog


The goal here is to represent each spouse candidate mention by a set of attributes or **_features_** which capture at least the key aspects of the mention, and then let a machine learning model learn how much each feature is correlated with our decision variable ('is this a spouse mention?').
For those who have worked with machine learning systems before, note that we are using a sparse storage representation-
you could think of a spouse candidate `(p1_id, p2_id)` as being represented by a vector of length `L = COUNT(DISTINCT feature)`, consisting of all zeros except for at the indexes specified by the rows with key `(p1_id, p2_id)`.

DeepDive includes an [automatic feature generation library, DDlib](http://deepdive.stanford.edu/gen_feats), which we will use here.
Although many state-of-the-art [applications](http://deepdive.stanford.edu/showcase/apps) have been built using purely DDlib-generated features, others can be used and/or added as well.
To use DDlib, we create a list of `ddlib.Word` objects, two `ddlib.Span` objects, and then use the function `get_generic_features_relation`, as shown in the following Python code for `udf/extract_spouse_features.py`:

In [34]:
%%file udf/extract_spouse_features.py
#!/usr/bin/env python
from deepdive import *
import ddlib

@tsj_extractor
@returns(lambda
        p1_id   = "text",
        p2_id   = "text",
        feature = "text",
    :[])
def extract(
        p1_id          = "text",
        p2_id          = "text",
        p1_begin_index = "int",
        p1_end_index   = "int",
        p2_begin_index = "int",
        p2_end_index   = "int",
        doc_id         = "text",
        sent_index     = "int",
        tokens         = "text[]",
        lemmas         = "text[]",
        pos_tags       = "text[]",
        ner_tags       = "text[]",
        dep_types      = "text[]",
        dep_parents    = "int[]",
    ):
    """
    Uses DDLIB to generate features for the spouse relation.
    """
    # Create a DDLIB sentence object, which is just a list of DDLIB Word objects
    sent = []
    for i,t in enumerate(tokens):
        sent.append(ddlib.Word(
            begin_char_offset=None,
            end_char_offset=None,
            word=t,
            lemma=lemmas[i],
            pos=pos_tags[i],
            ner=ner_tags[i],
            dep_par=dep_parents[i] - 1,  # Note that as stored from CoreNLP 0 is ROOT, but for DDLIB -1 is ROOT
            dep_label=dep_types[i]))

    # Create DDLIB Spans for the two person mentions
    p1_span = ddlib.Span(begin_word_id=p1_begin_index, length=(p1_end_index-p1_begin_index+1))
    p2_span = ddlib.Span(begin_word_id=p2_begin_index, length=(p2_end_index-p2_begin_index+1))

    # Generate the generic features using DDLIB
    for feature in ddlib.get_generic_features_relation(sent, p1_span, p2_span):
        yield [p1_id, p2_id, feature]


Overwriting udf/extract_spouse_features.py


In [35]:
!chmod +x udf/extract_spouse_features.py

Note that getting the input for this UDF requires joining the `person_mention` and `sentences` tables:


In [36]:
%%file -a app.ddlog

function extract_spouse_features over (
        p1_id          text,
        p2_id          text,
        p1_begin_index int,
        p1_end_index   int,
        p2_begin_index int,
        p2_end_index   int,
        doc_id         text,
        sent_index     int,
        tokens         text[],
        lemmas         text[],
        pos_tags       text[],
        ner_tags       text[],
        dep_types      text[],
        dep_tokens     int[]
    ) returns rows like spouse_feature
    implementation "udf/extract_spouse_features.py" handles tsj lines.

spouse_feature += extract_spouse_features(
    p1_id, p2_id, p1_begin_index, p1_end_index, p2_begin_index, p2_end_index,
    doc_id, sent_index, tokens, lemmas, pos_tags, ner_tags, dep_types, dep_tokens
) :-
    person_mention(p1_id, _, doc_id, sent_index, p1_begin_index, p1_end_index),
    person_mention(p2_id, _, doc_id, sent_index, p2_begin_index, p2_end_index),
    sentences(doc_id, sent_index, tokens, lemmas, pos_tags, ner_tags, _, dep_types, dep_tokens).


Appending to app.ddlog


Now, let's execute this UDF to get our features:

In [37]:
!deepdive redo spouse_feature

app.ddlog: updated since last `deepdive compile`
2016-07-08 05:47:23.445736 ‘run/LATEST.COMPILE’ -> ‘20160708/054723.403311000’
2016-07-08 05:47:23.445924 ‘run/RUNNING.COMPILE’ -> ‘20160708/054723.403311000’
2016-07-08 05:47:23.445947 Parsing DeepDive application (/ConfinedWater/spouse) to generate:
2016-07-08 05:47:23.445959  run/compiled/deepdive.conf
2016-07-08 05:47:23.445969   from app.ddlog
2016-07-08 05:47:23.924248  run/compiled/deepdive.conf.json
2016-07-08 05:47:24.074674 Performing sanity checks on run/compiled/deepdive.conf.json:
2016-07-08 05:47:24.103123  checking if input_extractors_well_defined
2016-07-08 05:47:24.103200  checking if input_schema_wellformed
2016-07-08 05:47:24.103644 Normalizing and adding built-in processes to the data flow to compile:
2016-07-08 05:47:24.104117  run/compiled/config-0.00-init_objects.json
2016-07-08 05:47:24.117400  run/compiled/config-0.01-parse_calibration.json
2016-07-08 05:47:24.130519  run/compiled/config-0.01-parse_schema.json
20

If we take a look at a sample of the extracted features, they will look roughly like the following:

In [38]:
!deepdive query '| 20 ?- spouse_feature(_, _, feature).'

                    feature                     
------------------------------------------------
 IS_INVERTED
 INV_WORD_SEQ_[Jeff Renz]
 INV_LEMMA_SEQ_[Jeff Renz]
 INV_NER_SEQ_[PERSON PERSON]
 INV_POS_SEQ_[NNP NNP]
 INV_W_LEMMA_L_1_R_1_[,]_[help]
 INV_W_NER_L_1_R_1_[O]_[O]
 INV_W_LEMMA_L_1_R_2_[,]_[help pour]
 INV_W_NER_L_1_R_2_[O]_[O O]
 INV_W_LEMMA_L_1_R_3_[,]_[help pour cement]
 INV_W_NER_L_1_R_3_[O]_[O O O]
 INV_W_LEMMA_L_2_R_1_[day ,]_[help]
 INV_W_NER_L_2_R_1_[DATE O]_[O]
 INV_W_LEMMA_L_2_R_2_[day ,]_[help pour]
 INV_W_NER_L_2_R_2_[DATE O]_[O O]
 INV_W_LEMMA_L_2_R_3_[day ,]_[help pour cement]
 INV_W_NER_L_2_R_3_[DATE O]_[O O O]
 INV_W_LEMMA_L_3_R_1_[the day ,]_[help]
 INV_W_NER_L_3_R_1_[DATE DATE O]_[O]
 INV_W_LEMMA_L_3_R_2_[the day ,]_[help pour]
(20 rows)



Now we have generated what looks more like the standard input to a machine learning problem—a set of objects, represented by sets of features, which we want to classify (here, as true or false mentions of a spousal relation).
However, we **don't have any supervised labels** (i.e., a set of correct answers) for a machine learning algorithm to learn from!
In most real world applications, a sufficiently large set of supervised labels is _not_ available.
With DeepDive, we take the approach sometimes referred to as _distant supervision_ or _data programming_, where we instead generate a **noisy set of labels using a mix of mappings from secondary datasets and other heuristic rules**.

## 2. Distant supervision with data and rules
In this section, we'll use _distant supervision_ (or '_data programming_') to provide a noisy set of labels for candidate relation mentions, with which we will train a machine learning model.

We'll describe two basic categories of approaches:

1. Mapping from secondary data for distant supervision
2. Using heuristic rules for distant supervision

Finally, we'll describe a simple majority-vote approach to resolving multiple labels per example, which can be implemented within DDlog.



### 2.1. Mapping from secondary data for distant supervision
First, we'll try using an external structured dataset of known married couples, from [DBpedia](http://wiki.dbpedia.org/), to distantly supervise our dataset.
We'll download the relevant data, and then map it to our candidate spouse relations.


#### Extracting and downloading the DBpedia data
Our goal is to first extract a collection of known married couples from DBpedia and then load this into the `spouses_dbpedia` table in our database.
To extract known married couples, we use the DBpedia dump present in [Google's BigQuery platform](https://bigquery.cloud.google.com).
First we extract the URI, name and spouse information from the DBpedia `person` table records in BigQuery for which the field `name` is not NULL.
We use the following query:

```sql
SELECT URI,name, spouse
FROM [fh-bigquery:dbpedia.person]
where name <> "NULL"
```

We store the result of the above query in a local project table `dbpedia.validnames` and perform a self-join to obtain the pairs of married couples.

```sql
SELECT t1.name, t2.name
FROM [dbpedia.validnames] AS t1
JOIN EACH [dbpedia.validnames] AS t2
ON t1.spouse = t2.URI
```

The output of the above query is stored in a new table named `dbpedia.spouseraw`.
Finally, we use the following query to remove symmetric duplicates.

```sql
SELECT p1, p2
FROM (SELECT t1_name as p1, t2_name as p2 FROM [dbpedia.spouseraw]),
     (SELECT t2_name as p1, t1_name as p2 FROM [dbpedia.spouseraw])
WHERE p1 < p2
```

The output of this query is stored in a local file.
The file contains duplicate rows (BigQuery does not support `distinct`).
It also contains noisy rows where the name field contains a string where the given name family name and multiple aliases were concatenated and reported in a string including the characters `{` and `}`.
Using the Unix commands `sed`, `sort` and `uniq` we first remove the lines containing characters `{` and `}` and then duplicate entries.
This results in an input file `spouses_dbpedia.csv` containing 6,126 entries of married couples.

*Note that we made this [`spouses_dbpedia.csv` available for download from GitHub](https://github.com/HazyResearch/deepdive/blob/master/examples/spouse/input/spouses_dbpedia.csv.bz2), so you don't have to repeat the above process.*

#### Loading DBpedia data to database

To load the known married couples data into DeepDive, we first declare the schema in DDlog:

In [39]:
%%file -a app.ddlog

## Distant Supervision ########################################################

spouses_dbpedia(
    person1_name text,
    person2_name text
).

Appending to app.ddlog


Notice that we can easily load the data in `spouses_dbpedia.csv` data to the table we just declared if we follow DeepDive's convention of organizing input data under `input/` directory.
The input file name simply needs to start with the target database table name.
Let's download the file from GitHub to `input/spouses_dbpedia.csv.bz2` under our application:

In [40]:
!cd input && curl -RLO "https://github.com/HazyResearch/deepdive/raw/master/examples/spouse/input/spouses_dbpedia.csv.bz2"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   174  100   174    0     0    176      0 --:--:-- --:--:-- --:--:--   176
100 77463  100 77463    0     0  69963      0  0:00:01  0:00:01 --:--:-- 69963


Then execute this command to load it into the database:

In [41]:
!deepdive redo spouses_dbpedia

app.ddlog: updated since last `deepdive compile`
2016-07-08 05:47:33.925380 ‘run/LATEST.COMPILE’ -> ‘20160708/054733.885591000’
2016-07-08 05:47:33.925594 ‘run/RUNNING.COMPILE’ -> ‘20160708/054733.885591000’
2016-07-08 05:47:33.925612 Parsing DeepDive application (/ConfinedWater/spouse) to generate:
2016-07-08 05:47:33.925623  run/compiled/deepdive.conf
2016-07-08 05:47:33.925633   from app.ddlog
2016-07-08 05:47:34.464689  run/compiled/deepdive.conf.json
2016-07-08 05:47:34.615834 Performing sanity checks on run/compiled/deepdive.conf.json:
2016-07-08 05:47:34.645420  checking if input_extractors_well_defined
2016-07-08 05:47:34.645479  checking if input_schema_wellformed
2016-07-08 05:47:34.645766 Normalizing and adding built-in processes to the data flow to compile:
2016-07-08 05:47:34.646137  run/compiled/config-0.00-init_objects.json
2016-07-08 05:47:34.659560  run/compiled/config-0.01-parse_calibration.json
2016-07-08 05:47:34.672942  run/compiled/config-0.01-parse_schema.json
20

Now the database should include tuples that look like the following:


In [42]:
!deepdive query '| 20 ?- spouses_dbpedia(name1, name2).'

        name1         |                  name2                  
----------------------+-----------------------------------------
 20th Earl of Arundel | Anne Howard Countess of Arundel
 Aadesh Shrivastava   | Vijayta Pandit
 Aafia Siddiqui       | Amjad Mohammed Khan
 A. A. Gill           | Amber Rudd
 Aamir Ali Malik      | Sanjeeda Shaikh
 Aamir Khan           | Kiran Rao
 Aarón Díaz           | Kate del Castillo
 Aaron Hotchner       | Beth Clemmons
 Aaron Spelling       | Carolyn Jones
 Aaron Staton         | Connie Fletcher
 Aarti Bajaj          | Anurag Kashyap
 Abbas                | Erum Ali
 Abbas Tyrewala       | Pakhi Tyrewala
 Abbe Lane            | Xavier Cugat
 Abbie G. Rogers      | Henry Huttleston Rogers
 Abby Jimenez         | Ramon Jimenez Jr.
 Abby Lockhart        | Luka Kovač
 Abby McDeere         | Mitch McDeere
 Abdel Hakim Amer     | Berlenti Abdul Hamid  برلنتي عبد الحميد
 Abdoulaye Wade       | Viviane Wade
(20 rows)



#### Supervising spouse candidates with DBpedia data
First we'll declare a new table where we'll store the labels (referring to the spouse candidate mentions), with an integer value (`True=1, False=-1`) and a description (`rule_id`):


In [43]:
%%file -a app.ddlog

spouse_label(
    p1_id   text,
    p2_id   text,
    label   int,
    rule_id text
).

Appending to app.ddlog


Let's put all the spouse candidate mentions with a `NULL` label.  This is just for simplifying some steps later:

In [44]:
%%file -a app.ddlog

# make sure all pairs in spouse_candidate are considered as unsupervised examples
spouse_label(p1,p2, 0, NULL) :-
    spouse_candidate(p1, _, p2, _).


Appending to app.ddlog


Next we'll implement a simple distant supervision rule which labels any spouse mention candidate with a pair of names appearing in DBpedia as true:

In [45]:
%%file -a app.ddlog

# distant supervision using data from DBpedia
spouse_label(p1,p2, 1, "from_dbpedia") :-
    spouse_candidate(p1, p1_name, p2, p2_name),
    spouses_dbpedia(n1, n2),
    [ lower(n1) = lower(p1_name), lower(n2) = lower(p2_name) ;
      lower(n2) = lower(p1_name), lower(n1) = lower(p2_name) ].


Appending to app.ddlog


It should be noted that there are many clear ways in which this rule could be improved (fuzzy matching, more restrictive conditions, etc.), but this serves as an example of one major type of distant supervision rule.

### 2.2. Using heuristic rules for distant supervision
We can also create a supervision rule which does not rely on any secondary structured dataset like DBpedia, but instead just uses some heuristic.
We set up a DDlog function, `supervise`, which uses a UDF containing several heuristic rules over the mention and sentence attributes:



In [46]:
%%file -a app.ddlog

# supervision by heuristic rules in a UDF
function supervise over (
        p1_id text, p1_begin int, p1_end int,
        p2_id text, p2_begin int, p2_end int,
        doc_id         text,
        sentence_index int,
        sentence_text  text,
        tokens         text[],
        lemmas         text[],
        pos_tags       text[],
        ner_tags       text[],
        dep_types      text[],
        dep_tokens     int[]
    ) returns (
        p1_id text, p2_id text, label int, rule_id text
    )
    implementation "udf/supervise_spouse.py" handles tsj lines.

spouse_label += supervise(
    p1_id, p1_begin, p1_end,
    p2_id, p2_begin, p2_end,
    doc_id, sentence_index,
    tokens, lemmas, pos_tags, ner_tags, dep_types, dep_token_indexes
) :-
    spouse_candidate(p1_id, _, p2_id, _),
    person_mention(p1_id, p1_text, doc_id, sentence_index, p1_begin, p1_end),
    person_mention(p2_id, p2_text,      _,              _, p2_begin, p2_end),
    sentences(
        doc_id, sentence_index,
        tokens, lemmas, pos_tags, ner_tags, _, dep_types, dep_token_indexes
    ).


Appending to app.ddlog


The Python UDF named [`udf/supervise_spouse.py`](udf/supervise_spouse.py) contains several heuristic rules:

* Candidates with person mentions that are too far apart in the sentence are marked as false.
* Candidates with person mentions that have another person in between are marked as false.
* Candidates with person mentions that have words like "wife" or "husband" in between are marked as true.
* Candidates with person mentions that have "and" in between and "married" after are marked as true.
* Candidates with person mentions that have familial relation words in between are marked as false.


In [47]:
%%file udf/supervise_spouse.py
#!/usr/bin/env python
from deepdive import *
import random
from collections import namedtuple

SpouseLabel = namedtuple('SpouseLabel', 'p1_id, p2_id, label, type')

@tsj_extractor
@returns(lambda
        p1_id   = "text",
        p2_id   = "text",
        label   = "int",
        rule_id = "text",
    :[])
# heuristic rules for finding positive/negative examples of spouse relationship mentions
def supervise(
        p1_id="text", p1_begin="int", p1_end="int",
        p2_id="text", p2_begin="int", p2_end="int",
        doc_id="text", sentence_index="int",
        tokens="text[]", lemmas="text[]", pos_tags="text[]", ner_tags="text[]",
        dep_types="text[]", dep_token_indexes="int[]",
    ):

    # Constants
    MARRIED = frozenset(["wife", "husband"])
    FAMILY = frozenset(["mother", "father", "sister", "brother", "brother-in-law"])
    MAX_DIST = 10

    # Common data objects
    p1_end_idx = min(p1_end, p2_end)
    p2_start_idx = max(p1_begin, p2_begin)
    p2_end_idx = max(p1_end,p2_end)
    intermediate_lemmas = lemmas[p1_end_idx+1:p2_start_idx]
    intermediate_ner_tags = ner_tags[p1_end_idx+1:p2_start_idx]
    tail_lemmas = lemmas[p2_end_idx+1:]
    spouse = SpouseLabel(p1_id=p1_id, p2_id=p2_id, label=None, type=None)

    # Rule: Candidates that are too far apart
    if len(intermediate_lemmas) > MAX_DIST:
        yield spouse._replace(label=-1, type='neg:far_apart')

    # Rule: Candidates that have a third person in between
    if 'PERSON' in intermediate_ner_tags:
        yield spouse._replace(label=-1, type='neg:third_person_between')

    # Rule: Sentences that contain wife/husband in between
    #         (<P1>)([ A-Za-z]+)(wife|husband)([ A-Za-z]+)(<P2>)
    if len(MARRIED.intersection(intermediate_lemmas)) > 0:
        yield spouse._replace(label=1, type='pos:wife_husband_between')

    # Rule: Sentences that contain and ... married
    #         (<P1>)(and)?(<P2>)([ A-Za-z]+)(married)
    if ("and" in intermediate_lemmas) and ("married" in tail_lemmas):
        yield spouse._replace(label=1, type='pos:married_after')

    # Rule: Sentences that contain familial relations:
    #         (<P1>)([ A-Za-z]+)(brother|stster|father|mother)([ A-Za-z]+)(<P2>)
    if len(FAMILY.intersection(intermediate_lemmas)) > 0:
        yield spouse._replace(label=-1, type='neg:familial_between')


Overwriting udf/supervise_spouse.py


In [48]:
!chmod +x udf/supervise_spouse.py

Note that the rough theory behind this approach is that we don't need high-quality (e.g., hand-labeled) supervision to learn a high quality model.
Instead, using statistical learning, we can in fact recover high-quality models from a large set of low-quality or **_noisy_** labels.


### 2.3. Resolving multiple labels per example with majority vote
Finally, we implement a very simple majority vote procedure, all in DDlog, for resolving scenarios where a single spouse candidate mention has multiple conflicting labels.
First, we sum the labels (which are all -1, 0, or 1):



In [49]:
%%file -a app.ddlog

# resolve multiple labels by majority vote (summing the labels in {-1,0,1})
spouse_label_resolved(p1_id, p2_id, SUM(vote)) :-
    spouse_label(p1_id, p2_id, vote, rule_id).


Appending to app.ddlog


Then, we simply threshold and add these labels to our decision variable table `has_spouse` (see next section for details here):


In [50]:
%%file -a app.ddlog

# assign the resolved labels for the spouse relation
has_spouse(p1_id, p2_id) = if l > 0 then TRUE
                      else if l < 0 then FALSE
                      else NULL end :- spouse_label_resolved(p1_id, p2_id, l).

Appending to app.ddlog


Once again, to execute all of the above, just run the following command:

In [51]:
!deepdive redo has_spouse

app.ddlog: updated since last `deepdive compile`
2016-07-08 05:47:37.903616 ‘run/LATEST.COMPILE’ -> ‘20160708/054737.862339000’
2016-07-08 05:47:37.903777 ‘run/RUNNING.COMPILE’ -> ‘20160708/054737.862339000’
2016-07-08 05:47:37.903793 Parsing DeepDive application (/ConfinedWater/spouse) to generate:
2016-07-08 05:47:37.903804  run/compiled/deepdive.conf
2016-07-08 05:47:37.903814   from app.ddlog
2016-07-08 05:47:38.514769  run/compiled/deepdive.conf.json
2016-07-08 05:47:38.686931 Performing sanity checks on run/compiled/deepdive.conf.json:
2016-07-08 05:47:38.715727  checking if input_extractors_well_defined
2016-07-08 05:47:38.715778  checking if input_schema_wellformed
2016-07-08 05:47:38.716036 Normalizing and adding built-in processes to the data flow to compile:
2016-07-08 05:47:38.716416  run/compiled/config-0.00-init_objects.json
2016-07-08 05:47:38.731609  run/compiled/config-0.01-parse_calibration.json
2016-07-08 05:47:38.744980  run/compiled/config-0.01-parse_schema.json
20

Recall that `deepdive do` will execute all upstream tasks as well, so this will execute all of the previous steps!

Now, we can take a brief look at how many candidates are supervised by different rules, which will look something like the table below.
Obviously, the counts will vary depending on your input corpus.

In [52]:
!deepdive query 'rule, @order_by COUNT(1) ?- spouse_label(p1,p2, label, rule).'

           rule           | COUNT(1) 
--------------------------+----------
 neg:familial_between     |       26
 pos:wife_husband_between |       49
 neg:third_person_between |      176
 neg:far_apart            |      241
                          |      640
(5 rows)



## 3. Learning and inference: model specification
Now, we need to specify the actual model that DeepDive will perform learning and inference over.
At a high level, this boils down to specifying three things:

1. What are the _variables_ of interest that we want DeepDive to predict for us?

2. What are the _features_ for each of these variables?

3. What are the _connections_ between the variables?

One we have specified the model in this way, DeepDive will _learn_ the parameters of the model (the weights of the features and potentially the connections between variables), and then perform _statistical inference_ over the learned model to determine the probability that each variable of interest is true.

For more advanced users: we are specifying a _factor graph_ where the features are unary factors, and then using SGD and Gibbs sampling for learning and inference.
Further technical detail is available [here](#).


### 3.1. Specifying prediction variables
In our case, we have one variable to predict per spouse candidate mention, namely, **is this mention actually indicating a spousal relation or not?**
In other words, we want DeepDive to predict the value of a Boolean variable for each spouse candidate mention, indicating whether it is true or not.
Recall that we started this tutorial with specifying this at the beginning of [`app.ddlog`](app.ddlog) as follows:

```ddlog
has_spouse?(
    p1_id text,
    p2_id text
).
```

DeepDive will predict not only the value of these variables, but also the marginal probabilities, i.e., the confidence level that DeepDive has for each individual prediction.

### 3.2. Specifying features
Next, we indicate (i) that each `has_spouse` variable will be connected to the features of the corresponding `spouse_candidate` row, (ii) that we wish DeepDive to learn the weights of these features from our distantly supervised data, and (iii) that the weight of a specific feature across all instances should be the same, as follows:


In [53]:
%%file -a app.ddlog

## Inference Rules ############################################################
 
# Features
@weight(f)
has_spouse(p1_id, p2_id) :-
    spouse_feature(p1_id, p2_id, f).


Appending to app.ddlog


### 3.3. Specifying connections between variables
Finally, we can specify dependencies between the prediction variables, with either learned or given weights.
Here, we'll specify two such rules, with fixed (given) weights that we specify.
First, we define a _symmetry_ connection, namely specifying that if the model thinks a person mention `p1` and a person mention `p2` indicate a spousal relationship in a sentence, then it should also think that the reverse is true, i.e., that `p2` and `p1` indicate one too:

In [54]:
%%file -a app.ddlog

# Inference rule: Symmetry
@weight(3.0)
has_spouse(p1_id, p2_id) => has_spouse(p2_id, p1_id) :-
    TRUE.


Appending to app.ddlog


Next, we specify a rule that the model should be strongly biased towards finding one marriage indication per person mention.
We do this inversely, using a negative weight, as follows:

In [55]:
%%file -a app.ddlog

# Inference rule: Only one marriage
@weight(-1.0)
has_spouse(p1_id, p2_id) => has_spouse(p1_id, p3_id) :-
    TRUE.


Appending to app.ddlog


### 3.4. Performing learning and inference

Finally, to perform learning and inference using the specified model, we need to run the following command:


In [56]:
!deepdive redo probabilities

app.ddlog: updated since last `deepdive compile`
2016-07-08 05:47:47.903059 ‘run/LATEST.COMPILE’ -> ‘20160708/054747.847273000’
2016-07-08 05:47:47.903259 ‘run/RUNNING.COMPILE’ -> ‘20160708/054747.847273000’
2016-07-08 05:47:47.903281 Parsing DeepDive application (/ConfinedWater/spouse) to generate:
2016-07-08 05:47:47.903294  run/compiled/deepdive.conf
2016-07-08 05:47:47.903304   from app.ddlog
2016-07-08 05:47:48.627625  run/compiled/deepdive.conf.json
2016-07-08 05:47:48.836799 Performing sanity checks on run/compiled/deepdive.conf.json:
2016-07-08 05:47:48.863952  checking if input_extractors_well_defined
2016-07-08 05:47:48.864003  checking if input_schema_wellformed
2016-07-08 05:47:48.864271 Normalizing and adding built-in processes to the data flow to compile:
2016-07-08 05:47:48.864629  run/compiled/config-0.00-init_objects.json
2016-07-08 05:47:48.878361  run/compiled/config-0.01-parse_calibration.json
2016-07-08 05:47:48.892115  run/compiled/config-0.01-parse_schema.json
20

This will ground the model based on the data in the database, learn the weights, infer the expectations or marginal probabilities of the variables in the model, and then load them back to the database.

Let's take a look at the probabilities inferred by DeepDive for the `has_spouse` variables.

In [57]:
!deepdive sql 'SELECT p1_id, p2_id, expectation FROM has_spouse_inference ORDER BY random() LIMIT 20'

                     p1_id                     |                     p2_id                     | expectation 
-----------------------------------------------+-----------------------------------------------+-------------
 328b5f1c-2b52-4eac-916c-c7983d4882a4_92_0_0   | 328b5f1c-2b52-4eac-916c-c7983d4882a4_92_2_2   |       0.033
 ddf3dfd2-cc21-46ca-a0e7-bd66eeb6bec6_9_1_1    | ddf3dfd2-cc21-46ca-a0e7-bd66eeb6bec6_9_4_4    |       0.021
 d6880afb-7fcb-4576-9d17-cedd343677f9_29_20_20 | d6880afb-7fcb-4576-9d17-cedd343677f9_29_36_36 |       0.004
 3deea828-e3a1-4c4d-9a90-482cabc020d8_3_19_19  | 3deea828-e3a1-4c4d-9a90-482cabc020d8_3_25_25  |        0.03
 9662058b-fca5-4771-8058-c7fd7bd548a3_67_15_15 | 9662058b-fca5-4771-8058-c7fd7bd548a3_67_26_27 |       0.007
 9662058b-fca5-4771-8058-c7fd7bd548a3_68_6_6   | 9662058b-fca5-4771-8058-c7fd7bd548a3_68_8_9   |        0.01
 0d07e617-00d4-4866-aee2-0ae197ae366f_19_16_16 | 0d07e617-00d4-4866-aee2-0ae197ae366f_19_9_9   |       0.005
 ac6492fb-c67d-49